# Visualize data in Neuroglancer

## 1. Import packages and starting neuroglancer server
By running the cell below, you will get a link as output (using port 7010 by default, but you can change it). 
After you run the full notebook, at this link you will be able to see the segmentations and microscopy images.

Documentation on [neuroglancer](https://github.com/google/neuroglancer) is kind of scarse, but if you click on the question mark symbol in the top-right corner of the interface, you will see all the shortcuts and commands you can use to navigate the interface:

- Neuroglancer is a software to visualize 3d data and segmentations, and in this case all datasets/segmentations are stacked along a 3rd dimension, so we only care about looking data across xy dimensions (not across xz or yz).
- By default in neuroglancer you see all three views, but you can maximize the xy view to the full screen and ignore the other ones (usually the quadrant on the bottom-right).
- By scrolling with your mouse, you can move through the third dimension (moving from one dataset to the next one). Instead, with Ctrl+Scroll you can zoom in and out.
- Layers are listed on the top-left of the interface. If you re-run the second cell of this notebook, microscopy layers will be updated in the viewer. If you instead re-run the third cell, you will update the segmentation layers. 
- You can hide or show a layer by clicking on it
- By right-clicking on a layer, you open a side-bar on the right: in the `Rendering` tab you find the opacity slider and you can also adjust the contrast of min-max values, similarly to Napari

![alt text](imagename.png "Title")

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import argparse
from itertools import repeat


from functools import partial

from segmfriends.utils.various import parse_data_slice
from segmfriends.utils import various as sgm_utils
import time
import h5py

import neuroglancer

# if 'viewer' not in globals():
neuroglancer.set_server_bind_address(bind_port=7010)
viewer = neuroglancer.Viewer()

print(viewer)

from PIL import Image
from segmfriends.utils.various import writeHDF5, readHDF5, get_hdf5_inner_paths


http://127.0.0.1:7010/v/de511d0ad26555f2e16f317b3117d90fbe73fc3c/


## 2. Load microscopy data in neuroglanger viewer
- Below, indicate the path to the zarr file specified in your config file (parameter named `preprocessing/data_zarr_group`).
- The `layer_specs` list has the following format: `(NAME_OF_CHANNEL_STORED_IN_ZARR_FILE, "image", NAME_LAYER_SHOWN_IN_NEUROGLANCER)`
- You should use the same channel names defined in the config file, and you can use any layer name you prefer 

In [2]:
from segmfriends.utils.neuroglancer import visualize_zarr_in_neuroglancer, visualize_zarr_in_neuroglancer_as_numpy
dimensions=neuroglancer.CoordinateSpace(
                scales=[1, 1, 1],
                units=['', '', ''],
                names=['z', 'y', 'x'])



with viewer.txn() as s:
    visualize_zarr_in_neuroglancer_as_numpy(s,
                                            "/scratch/bailoni/projects/cellpose_inference_projects/test_segm_tool/data_v1.zarr",
                                            coordinate_space=dimensions,
                                            layers_specs=[("BF", "image", "BF"),
                                                         ("DAPI", "image", "DAPI"),]
                                  )



## 3. Load Segmentations in neuroglancer viewer
- As path to segmentations, below you should indicate the following: `/path/to/your/RUN_DIRECTORY/` + `cellpose_predictions/predictions_collected.zarr`
- The `layers_specs` list follows the same format, but now we should use `segm` as type of layer, to make sure that the zarr data will be visualized as segmentation layer
- The first element of each tuple indicates the zarr_channel name of the segmentation, which will follow as usual the naming scheme of `"cellpose_model_name"+"_diamEst` if the diameter was estimated, otherwise it will have the `_noDiamEst` suffix.

In [3]:
with viewer.txn() as s:
    visualize_zarr_in_neuroglancer_as_numpy(s,
                                   "/scratch/bailoni/projects/cellpose_inference_projects/test_segm_tool/test_run/cellpose_predictions/predictions_collected.zarr",
                                   coordinate_space=dimensions,
                                   layers_specs=[
                                        ("cyto2_diamEst", "segm", "cyto2_diamEst"),
#                                        ("cyto2_noDiamEst", "segm", "trained_model_noDiamEst"),
#                                       ("trained_model_noDiamEst", "segm", "trained"),
                                                ],
                                  )


